# Setup 

In [1]:
import argparse
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
torch.__version__

'0.4.0'

## Args and random seed

In [3]:
class args_c():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 2
        self.lr = 0.01 
        self.momentum = 0.5
        self.seed = 1
        self.log_interval = 1
        
args = args_c()

In [4]:
torch.manual_seed(args.seed); #random seed

## Dataloaders

In [5]:
transform = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])

MNIST_train = datasets.MNIST(r'D:\Data_sets/MNIST', train=True, download=True,
                   transform=transform)

MNIST_test = datasets.MNIST(r'D:\Data_sets/MNIST', train=False, download=True,
                   transform=transform)

In [6]:
train_loader = torch.utils.data.DataLoader(MNIST_train, 
                                           batch_size=args.batch_size, 
                                           shuffle=True, 
                                           pin_memory=True)

test_loader = torch.utils.data.DataLoader(MNIST_test,
                                          batch_size=args.test_batch_size, 
                                          shuffle=True, 
                                          pin_memory=True)

## Network definition

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Net().cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

## Train

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
def train(args, model, train_loader, optimizer, epoch):
    
    model.train() #for dropout
    
    running_loss = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = Variable(data.cuda()), Variable(target.cuda())
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = criterion(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()

    if epoch % args.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), running_loss / len(train_loader.dataset)))

In [11]:
def test(args, model, test_loader):
    
    model.eval() #for dropout
    
    test_loss = 0
    
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            
            data, target = data.cuda(), target.cuda()
            
            output = model(data)
            
            test_loss += criterion(output, target).item()
            
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
test(args, model, test_loader)


Test set: Average loss: 0.0023, Accuracy: 1137/10000 (11%)



In [13]:
def run():
    for epoch in range(1, args.epochs + 1):
        train(args, model, train_loader, optimizer, epoch)
        test(args, model, test_loader)

In [14]:
run()

Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.014433

Test set: Average loss: 0.0002, Accuracy: 9402/10000 (94%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.006059

Test set: Average loss: 0.0001, Accuracy: 9592/10000 (96%)

